In [2]:
import pandas as pd
df=pd.read_csv("dataset/dataset.csv")
df=df.fillna(0)

In [3]:
df=df.to_csv("./preprocessed/preprocessed2.csv",index=False)

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession.builder.master("local").appName("sample").getOrCreate()
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('E:/ongoing/spark/preprocessed/preprocessed2.csv')
df.take(1)
df.show()


AnalysisException: Path does not exist: file:/E:/ongoing/spark/preprocessed/preprocessed2.csv;

In [6]:
print("Rows= ",df.count())
print("Columns= ",len(df.columns))

AttributeError: 'NoneType' object has no attribute 'count'

In [7]:
'''check null values'''
from pyspark.sql.functions import isnull, when, count, col
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

AttributeError: 'NoneType' object has no attribute 'select'

In [8]:
print(df.count(),len(df.columns))
CONT_COLS_NAMES=[]
STRING_COLS_NAMES=[]
for i in df.dtypes:
    if i[1] == "string":
        STRING_COLS_NAMES.append(i[0])
    else:
        CONT_COLS_NAMES.append(i[0])
print(STRING_COLS_NAMES)
print(CONT_COLS_NAMES)

AttributeError: 'NoneType' object has no attribute 'count'

In [ ]:
'''label'''
from pyspark.ml.feature import StringIndexer
stage_1 = StringIndexer(inputCol= 'age', outputCol= 'age_index')
stage_1.setHandleInvalid("keep")
stage_2 = StringIndexer(inputCol= 'sex', outputCol= 'sex_index')
stage_2.setHandleInvalid("keep")
stage_3 = StringIndexer(inputCol= 'cp', outputCol= 'cp_index')
stage_3.setHandleInvalid("keep")
stage_4 = StringIndexer(inputCol= 'trestbps', outputCol= 'trestbps_index')
stage_4.setHandleInvalid("keep")
stage_5 = StringIndexer(inputCol= 'chol', outputCol= 'chol_index')
stage_5.setHandleInvalid("keep")
stage_6 = StringIndexer(inputCol= 'fbs', outputCol= 'fbs_index')
stage_6.setHandleInvalid("keep")
stage_7 = StringIndexer(inputCol= 'restecg', outputCol= 'restecg_index')
stage_7.setHandleInvalid("keep")
stage_8= StringIndexer(inputCol= 'thalach', outputCol= 'thalach_index')
stage_8.setHandleInvalid("keep")


In [ ]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
import os
stage_14 = OneHotEncoder(inputCols=[
                                stage_1.getOutputCol(),stage_2.getOutputCol(), 
                                stage_3.getOutputCol(),stage_4.getOutputCol(),
                                stage_5.getOutputCol(),stage_6.getOutputCol(),
                                stage_7.getOutputCol(),stage_8.getOutputCol(),
                                
                               
                                ],
                                outputCols= [
                                'age_encoded','sex_encoded',
                                'cp_encoded','trestbps_encoded',
                                'chol_encoded','fbs_encoded',
                                'restecg_encoded','thalach_encoded',
                             
                                
                                ])

stage_15 = VectorAssembler(inputCols=[
                                'age_encoded','sex_encoded',
                                'cp_encoded','trestbps_encoded',
                                'chol_encoded','fbs_encoded',
                                'restecg_encoded','thalach_encoded',
                                'exang','oldpeak','slope','ca','thal'
                                ],outputCol='features')

from pyspark.ml import Pipeline
regression_pipeline = Pipeline(stages= [stage_1, stage_2, stage_3, stage_4, stage_5,
                                        stage_6,stage_7,stage_8,stage_14,stage_15])
ppmodel = regression_pipeline.fit(df)
data = ppmodel.transform(df)
print(data.select('features').show(1))


In [ ]:
'''Standard Scaling'''
from pyspark.ml.feature import StandardScaler
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
scalar_fit=standardscaler.fit(data)
data=scalar_fit.transform(data)

data.select("features","Scaled_features").show(5)

In [ ]:
train, test = data.randomSplit([0.8, 0.2], seed=12345)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'Scaled_features', labelCol = 'target')
dtModel = dt.fit(train)
dt_predictions = dtModel.transform(test)
dt_predictions.select('target', 'prediction', 'probability').show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='target', 
    predictionCol='prediction', 
    metricName='accuracy')
accuracy = evaluator.evaluate(dt_predictions)
print('Train Accuracy = ', accuracy)
DT_SC=accuracy*100

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'Scaled_features', labelCol = 'target', maxIter=10)
lrModel = lr.fit(train)
lr_predictions = lrModel.transform(test)
print(lr_predictions)
lr_predictions.select('target', 'prediction', 'probability').show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='target', 
    predictionCol='prediction', 
    metricName='accuracy')
accuracy = evaluator.evaluate(lr_predictions)
print('Train Accuracy = ', accuracy)
LR_SC=accuracy*100

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'Scaled_features', labelCol = 'target')
rfModel = rf.fit(train)
rf_predictions = rfModel.transform(test)

rf_predictions.select('target', 'prediction', 'probability').show(8)

In [ ]:
#train, test = data.randomSplit([0.8, 0.2], seed=12345)


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='target', 
    predictionCol='prediction', 
    metricName='accuracy')
accuracy = evaluator.evaluate(rf_predictions)
model=accuracy
print('Train Accuracy = ', accuracy)
RF_SC=accuracy*100

In [ ]:
'''Feature_Importance'''
print(rfModel.featureImportances)
import pandas as pd

def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

ExtractFeatureImp(rfModel.featureImportances, train, "features").head(10)

In [ ]:
'''model_testing'''

print(test.count(),len(test.columns))
rf_predictions.select('target', 'prediction', 'probability').show()

a=rf_predictions.select('target', 'prediction', 'probability').toPandas()
print(a)
a.to_csv("./output/RANFOR_test.csv")

In [ ]:
'''bias'''
predictions = rfModel.transform(train)

print(train.count(),len(train.columns))

predictions.select('target', 'prediction', 'probability').show()

a=predictions.select('target', 'prediction', 'probability').toPandas()
print(a)
a.to_csv("./output/RANFOR_Train.csv")

In [ ]:
'''confusion matrix'''
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType


In [ ]:
pred_and_labels = predictions.select(['prediction','target']).withColumn('target',F.col('target').cast(FloatType())).orderBy('prediction')
pred_and_labels=pred_and_labels.select(['prediction','target'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
height = [RF_SC,DT_SC,LR_SC]
bars = ( 'RF', 'DT','LR')
x_pos = np.arange(len(bars))
plt.bar(x_pos, height, color=['#E32227', '#267055', '#050A30'])
plt.xticks(x_pos, bars)
plt.show()

In [ ]:
import pickle

In [ ]:
with open('./model/heart.pkl','wb') as f:
    pickle.dump(model,f)
    f.close()

In [ ]:
model = pickle.load(open('./model/heart.pkl', 'rb'))